<a href="https://colab.research.google.com/github/angel539/Python-Notebooks/blob/main/WebScrapping_quesos_Carrefour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando las librerias:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# DataFrame vacío.
df_quesos = pd.DataFrame(columns = ['Nombre','Formato','Precio_Kilo','Precio_Normal','Precio_Descontado', 'Descuento'])

### Haciendo Web Scrapping con BeautifulSoup.

In [3]:
# Paginacion
for num in range(0, 583, 24):
    url              = "https://www.carrefour.es/supermercado/el-mercado/quesos/todos-los-quesos/N-ndzi6a/c?No=" + str(num) + "&Nr%3DAND(product.shopCodes%3A004320%2Cproduct.salepointWithActivePrice_004320%3A1%2COR(product.siteId%3AbasicSite))OR)"
    response         = requests.get(url,  headers={'User-Agent': 'Mozilla/5.0'})
    soup             = BeautifulSoup(response.text, "html.parser")

    todos_los_quesos = soup.find_all("article", class_="product-card-item")

    # Recorrer por item.
    for i in range(len(todos_los_quesos)-1):
        try:
          # Resolver los valores que voy a guardar en el DataFrame.
          nombre                  = todos_los_quesos[i].find("p", class_="title-product").text.replace("\n","")
          formato_precio          = todos_los_quesos[i].find("p", class_="format-price").text.replace("\n","").replace("\xa0","")
          formato_precio_spplited = formato_precio.split(" |")
          formato                 = formato_precio_spplited[0]
          precio_kilo             = formato_precio_spplited[1].replace('€/kg', '').replace('€/ud','')     
          precio_normal           = todos_los_quesos[i].find("span", class_='price').text.split('\xa0')[0]
          precio_descontado       = precio_normal
          descontado              = False
        except:
          precio_normal           = todos_los_quesos[i].find("span", class_='strike-price').text.split('\n')[2].split('\xa0')[0]
          precio_descontado       = todos_los_quesos[i].find('span', class_='price-less').text.split('\n')[1].split('\xa0')[0]      
          descontado              = True
        finally:
            # Guardar en el dataframe.
            new_row = {
                'Nombre'            : nombre,
                'Formato'           : formato,
                'Precio_Kilo'       : float(precio_kilo.replace(',', '.')),
                'Precio_Normal'     : float(precio_normal.replace(',', '.')),
                'Precio_Descontado' : float(precio_descontado.replace(',', '.')),
                'Descuento'         : descontado
            }
            df_quesos = df_quesos.append(new_row, ignore_index=True)

In [4]:
# Para comprobar los tipos.
# df_quesos.dtypes

### Guardando el dataframe en un CSV para no perder los datos.

In [5]:
df_quesos.to_csv('quesos.csv')